In [ ]:
import random as rdn
from copy import deepcopy 

In [ ]:
class formiga:
    def __init__(self, quantidadeRainhas, evaporacao):
        self.iteracao = 0
        self.custo = quantidadeRainhas
        self.rainhas = quantidadeRainhas
        self.valorEvaporacao = evaporacao
        
        self.caminho = [i for i in range(quantidadeRainhas)] 
        self.matrizFeromonio = [[1 for i in range(quantidadeRainhas)] for j in range(quantidadeRainhas)]
        self.matrizProbabilidade = [[0 for i in range(quantidadeRainhas)] for j in range(quantidadeRainhas)]
        
    def caminhoInicial(self):
        rdn.shuffle(self.caminho)
    
    def geraMatrizRainhas(self):
        self.matrizRainhas = [[0 for i in range(self.rainhas)] for j in range(self.rainhas)]
        
        for l, c in enumerate(self.caminho):
            self.matrizRainhas[l][c] = 1
    
    def calcularCusto(self):
        dicionario = {}
        for i in self.caminho:
            if i in dicionario:
                dicionario[i] = dicionario[i] + 1
            else:
                dicionario[i] = 1
            
        self.custo = int(self.custoDiagonal() / 2) + self.custoColuna()
    
    def maiorValorMatrizFeromonio(self):
        maiorValor = 0
        
        for linha in self.matrizFeromonio:
            maiorValorLinha = max(linha)
            if maiorValorLinha > maiorValor:
                maiorValor = maiorValorLinha
        
        return maiorValor
    
    
    def evaporarFeromonio(self):
        for linha in range(self.rainhas):
            for coluna in range(self.rainhas):
                if (coluna != self.caminho[linha]):
                    self.matrizFeromonio[linha][coluna] = round(self.matrizFeromonio[linha][coluna] - self.matrizFeromonio[linha][coluna] * self.valorEvaporacao, 3)
#                     self.matrizFeromonio[linha][coluna] = round(self.matrizFeromonio[linha][coluna] / self.maiorValorMatrizFeromonio(), 3)
    
    def custoDiagonal(self):
        somaDiagonal = 0 
        
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):

                if self.caminho[i]+(j-i) < len(self.caminho):
                    if self.caminho[i+(j-i)] == self.caminho[i]+(j-i):
                        somaDiagonal += 1
                if self.caminho[i]-(j-i) >= 0:
                    if self.caminho[i+(j-i)] == self.caminho[i]-(j-i):
                        somaDiagonal += 1

            for j in range(0,i):
                if self.caminho[i]+(j+1) < len(self.caminho):
                    if self.caminho[i-(j+1)] == self.caminho[i]+(j+1):
                        somaDiagonal += 1
                if self.caminho[i]-(j+1) >= 0:
                    if self.caminho[i-(j+1)] == self.caminho[i]-(j+1):
                        somaDiagonal += 1
        
        return somaDiagonal
    
    def custoColuna(self):
        custo = 0
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):
                if self.caminho[i] == self.caminho[j]:
                    custo += 1
        return custo
    
    def calcularMatrizFeromonio(self):
        self.calcularCusto()
        
        if (self.custo == 0):
            return self.caminho

        feromonio = round(1 / self.custo, 3)
        
        self.evaporarFeromonio()
        
        for linha, coluna in enumerate(self.caminho):
            self.matrizFeromonio[linha][coluna] = round(self.matrizFeromonio[linha][coluna] + feromonio, 3)
                
    def calcularMatrizProbabilidade(self):
        somas = [0 for i in range(self.rainhas)]
            
        for linha in self.matrizFeromonio:    
            for indice, coluna in enumerate(linha):
                somas[indice] += coluna

        for indiceLinha, linha in enumerate(self.matrizFeromonio):  
            for indiceColuna, coluna in enumerate(linha):
                self.matrizProbabilidade[indiceLinha][indiceColuna] = round(coluna / somas[indiceColuna], 3)
                              
    def novoCaminhoFormiga(self):
        i = 0

        for line in self.matrizProbabilidade:
            roleta = [0 for n in line]
            somaPeso = sum(line)
            sorteio = rdn.random()
            
            prob = [x*100/somaPeso for x in line]
            
            roleta[0] = (0, prob[0])
            for x in range(1, len(prob)):
                roleta[x] = (roleta[x-1][1], roleta[x-1][1] + prob[x])

            self.caminho[i] = self.escolherPosicaoArbitraria(roleta, sorteio)
            i += 1
    
    def escolherPosicaoArbitraria(self, roleta, sorteio):
        sorteio *= 100
        posicao = 0
        for minimo, maximo in roleta:
            
            if sorteio >= minimo and sorteio < maximo:
                return posicao
            
            posicao += 1
        
        return posicao
    
    def atualizarMatrizFeromonio(self, novaMatrizFeromonio):
        self.matrizFeromonio = novaMatrizFeromonio
    
    def debug(self):
        print('Iteracao', self.iteracao)
        print('Custo', self.custo)
        
        print('Rainhas')
        printMatriz(self.matrizRainhas)
        
        print('Feromonios')
        printMatriz(self.matrizFeromonio)
        
        print('Probabilidade')
        printMatriz(self.matrizProbabilidade)
    
    def prepararBusca(self):
        self.caminhoInicial()
                
    def encontrarUmCaminho(self):
        self.geraMatrizRainhas()
        self.calcularMatrizFeromonio()
        self.calcularMatrizProbabilidade()
        self.novoCaminhoFormiga()
        
    def retornarConflitos(self):
        return self.custo
    
    def retornarMatrizFeromonio(self):
        return self.matrizFeromonio

In [ ]:
def printMatriz(matriz):
    for linha in matriz:
        for coluna in linha:
            print(coluna, end='  ')
        print()
    print()

In [ ]:
def somaMatrizesFeromonio(matrizesFeromonio, rainhas, formigas):
    matrizFinal = [[0 for i in range(rainhas)] for j in range(rainhas)]
    
    for matriz in matrizesFeromonio:
        matrizFinal = somaMatriz(matrizFinal, matriz)
    
    for i in range(rainhas):
        for j in range(rainhas):
            matrizFinal[i][j] /= formigas
            
    return matrizFinal

In [ ]:
def somaMatriz(matrizAlvo, matrizFonte):
    ordem = len(matrizAlvo)
    for i in range(ordem):
        for j in range(ordem):
            matrizAlvo[i][j] += matrizFonte[i][j]
            
    return matrizAlvo

In [ ]:
def main(rainhas, numFormigas, valorEvaporacao):
    conflito = rainhas
    formigas = []
    matrizFeromonioCompartilhada = [[1 for i in range(rainhas)] for j in range(rainhas)]
    
    for i in range(numFormigas):
        formigas.append(formiga(rainhas, valorEvaporacao))
    
    for formigaIndividual in formigas:
        formigaIndividual.prepararBusca()
        
    while conflito > 0:
        conflitos = []
        matrizesFeromonio = []
        
        for formigaIndividual in formigas:
            formigaIndividual.atualizarMatrizFeromonio(deepcopy(matrizFeromonioCompartilhada))
            formigaIndividual.encontrarUmCaminho()
            
            conflitos.append(formigaIndividual.retornarConflitos())
            matrizesFeromonio.append(formigaIndividual.retornarMatrizFeromonio())
        
        conflito = min(conflitos)
        printMatriz(matrizFeromonioCompartilhada)
        matrizFeromonioCompartilhada = somaMatrizesFeromonio(matrizesFeromonio, rainhas, numFormigas)
        
        
    return formigas[conflitos.index(0)]

In [ ]:
formigaSolucao = main(6, 2, 0.001)
formigaSolucao.matrizRainhas